In [1]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import albumentations as albu
import segmentation_models as sm
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
from model import ModelCreator
from datapipeline import DatasetCreatorTest
import shutil

Segmentation Models: using `tf.keras` framework.


In [2]:
height = 512

In [3]:
model = tf.keras.models.load_model('bestmodel',compile=False)
model.compile('Adam', loss=sm.losses.bce_jaccard_loss, metrics=[sm.metrics.iou_score])

In [4]:
val_transforms = albu.Compose([
    albu.Resize(height, height),
    albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
modelcreator = ModelCreator()
testdataset = DatasetCreatorTest(modelcreator.getpreprocess(),val_transforms)

In [8]:
testdata = testdataset('cropped/*png',batch_size=1)
len(testdata)

195

In [9]:
i = 0

if os.path.isdir('output2/'):
    shutil.rmtree('output2/')
os.makedirs('output2/')

for image in testdata.take(len(testdata)):
    mask = model.predict((tf.reshape(image,[1,height,height,3])))
    mask = tf.keras.utils.array_to_img(mask[0])
    img = tf.keras.utils.array_to_img(image[0])
    img = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2RGB)
    mask = cv2.cvtColor(np.array(mask),cv2.COLOR_GRAY2RGB)
    vis = np.concatenate((img, mask), axis=1)
    i+=1
    cv2.imwrite('output2/'+str(i)+'.png', vis)

1/1 [==============================] - 0s 124ms/step
